In [1]:
import sys
import os

debug = False

cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.append(cwd)

import xml.etree.ElementTree as etree
from modulosXMILE.Model import *

source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

DIR_XMILE       = 'lotka-volterra-nested/lotka-volterra-nested-2.xmile'
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()
models = root.findall(source_xmlns + 'model')
models_parsed = list(map(lambda x : Model(x, debug), models))
for model in models_parsed:
    print model.getName()

In [2]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
devsml_template_filename = 'template-devsml.xml'

# Clases DEVS

In [3]:
### TODO : Constant, Aux, Integrator, Ftot y Fpm podrian ser subclase de DEVSAtomicComponent
from modulosDEVS.DEVSCoupledComponent import *

topModel = models_parsed[0]
dm = DEVSCoupledComponent(topModel, models_parsed)

filenames = []
def traverse(dm):
    name = dm.getName()
    dst_filename = name
    
    # Acumulo todos los nombres para despues borrarlos
    filenames.append(name)
    
    ccs = dm.getDEVSCoupledComponents()
    ccs_names = []
    
    # Recursion
    for cc in ccs:
        traverse(cc)
        ccs_names.append(cc.getName())

    # Genero xml
    context = {
        'coupled_name' : name,
        'coupled_filenames' : ccs_names,
        'atomics'           : dm.getDEVSAtomicComponents(),
        'input_ports'       : dm.getDEVSInputPorts(),
        'output_ports'      : dm.getDEVSOutputPorts(),
        'internal_connections'        : dm.getDEVSInternalConnections(),
        'external_input_connections'  : dm.getDEVSExternalInputConnections(),
        'external_output_connections' : dm.getDEVSExternalOutputConnections()
    }
    coupled_xml = render_template(devsml_template_filename, context)
        
    # Cargo los 'include' del modelo
    from xml.etree import ElementTree as et
    from xml.etree  import ElementTree
    tree = et.fromstring(coupled_xml)
    includes = tree.findall('.//include')
    
    for include in includes:
        include_filename = include.get('filename')
        include_tree = et.parse(include_filename)

        for element in tree.iter():
            if element.tag == 'include' and element.get('filename') == include_filename:
                tree.find('components').append(include_tree.getroot())
    
    tree2 = et.tostring(tree)
    with open(dst_filename, 'wb') as f:
        f.write(tree2)

traverse(dm)

# Pretty print
import vkbeautify as vkb
with open('DEVS_COUPLED_top', 'r') as xml_file_new:
    parser = etree.XMLParser(encoding="utf-8")
    xml_tree_new = etree.parse(xml_file_new, parser=parser)
root = xml_tree_new.getroot()

# Borro includes
for elem in root.iter():
    for child in list(elem):
        if child.tag == 'include':
            elem.remove(child)
x = etree.tostring(root)
pretty_xml = vkb.xml(x) 
vkb.xml(x, 'top.xml')

# Borro archivos que ya no necesito
for filename in filenames:
    os.remove(filename)